# ANALISI DATI DI TEST

#### Autore: Enrico Brunelli

#### Data ultima modifica: 19/11/2024

### Descrizione:
 I test effettuati sono stati condotti variando i nodi di rete che generano traffico. Tali nodi sono stati generati da lato client connettendo tre distinti moduli Quectel EVB connessi al PC MSI con Windows 11. In tal modo è stato possibile utilizzare la rete dei moduli contemporaneamente. Sono stati implementati tre client tramite il software iperf3. Lato server, sul PC Firecell che contiene il Core Network della rete 5G, sono stati configurati 3 server tramite iperf3 sono 3 distinte porte: 5201 6201 7201.
 Inoltre è stato analizzato l'andamento dei container tramite il comando *docket stats* durante la connessione di 6 distinti User Equipement rappresentati dai moduli Quectel. In tal contesto è stato anche analizzato e catturato il traffico sull'interfaccia *enp5s0* in modo da osservare l'andamento dei pacchetti durante la fase di registrazione sulla rete. 


In [70]:
# Librerie necessarie all'analisi dei dati e la relativa rappresentazione
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ntplib import NTPStats
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MultipleLocator

In [71]:
# Nomi dei file rispettivamente per Client e Server
file_client="test_2_19_11_2024/istanti_temporali_2024-11-19.csv"
file_server="test_2_19_11_2024/server_istanti_temporali_2024-11-19.csv"

df1 = pd.read_csv(file_client) #legge il file del client
df2 = pd.read_csv(file_server) #legge il file del server

# Converto le colonne del file del Client che contengono l'ntp object
print(df1)
print(df2)

      Protocollo  Traffico  client_send_timestamp  client_recv_timestamp
0            TCP         0           1.732030e+09           1.732030e+09
1            TCP         0           1.732030e+09           1.732030e+09
2            TCP         0           1.732030e+09           1.732030e+09
3            TCP         0           1.732030e+09           1.732030e+09
4            TCP         0           1.732030e+09           1.732030e+09
...          ...       ...                    ...                    ...
15966        TCP         0           1.732031e+09           1.732031e+09
15967        TCP         0           1.732031e+09           1.732031e+09
15968        TCP         0           1.732031e+09           1.732031e+09
15969        TCP         0           1.732031e+09           1.732031e+09
15970        TCP         0           1.732031e+09           1.732031e+09

[15971 rows x 4 columns]
      Tipo  server_send_timestamp  server_recv_timestamp
0      TCP           1.732030e+09        

In [72]:
# Unisco i risultati ottenuti lato client con quelli ottenuti lato server

df1['server_send_timestamp']=df2['server_send_timestamp']
df1['server_recv_timestamp']=df2['server_recv_timestamp']
print(df1)

      Protocollo  Traffico  client_send_timestamp  client_recv_timestamp  \
0            TCP         0           1.732030e+09           1.732030e+09   
1            TCP         0           1.732030e+09           1.732030e+09   
2            TCP         0           1.732030e+09           1.732030e+09   
3            TCP         0           1.732030e+09           1.732030e+09   
4            TCP         0           1.732030e+09           1.732030e+09   
...          ...       ...                    ...                    ...   
15966        TCP         0           1.732031e+09           1.732031e+09   
15967        TCP         0           1.732031e+09           1.732031e+09   
15968        TCP         0           1.732031e+09           1.732031e+09   
15969        TCP         0           1.732031e+09           1.732031e+09   
15970        TCP         0           1.732031e+09           1.732031e+09   

       server_send_timestamp  server_recv_timestamp  
0               1.732030e+09     

In [73]:
# Calcolo RTT e OTT 
delay_richiesta_ntp_avg=0.005
df1['RTT']=(df1['client_recv_timestamp']-df1['server_send_timestamp'])+(df1['server_recv_timestamp']-df1['client_send_timestamp'])-(delay_richiesta_ntp_avg*4)
df1['OTT_send']=df1['server_recv_timestamp']-df1['client_send_timestamp']-(delay_richiesta_ntp_avg*2)
df1['OTT_recv']=df1['client_recv_timestamp']-df1['server_send_timestamp']-(delay_richiesta_ntp_avg*2)
print(df1)

      Protocollo  Traffico  client_send_timestamp  client_recv_timestamp  \
0            TCP         0           1.732030e+09           1.732030e+09   
1            TCP         0           1.732030e+09           1.732030e+09   
2            TCP         0           1.732030e+09           1.732030e+09   
3            TCP         0           1.732030e+09           1.732030e+09   
4            TCP         0           1.732030e+09           1.732030e+09   
...          ...       ...                    ...                    ...   
15966        TCP         0           1.732031e+09           1.732031e+09   
15967        TCP         0           1.732031e+09           1.732031e+09   
15968        TCP         0           1.732031e+09           1.732031e+09   
15969        TCP         0           1.732031e+09           1.732031e+09   
15970        TCP         0           1.732031e+09           1.732031e+09   

       server_send_timestamp  server_recv_timestamp       RTT  OTT_send  \
0           

In [75]:

df1=df1[df1['RTT']<0.200]
df1=df1[df1['RTT']>0]
stats_rtt = df1.groupby(['Traffico'])['RTT'].agg(['mean', 'var','std','median','count','min','max'])
stats_ott_send = df1.groupby(['Traffico'])['OTT_send'].agg(['mean', 'var','std','median','count','min','max'])
stats_ott_recv = df1.groupby(['Traffico'])['OTT_recv'].agg(['mean', 'var','std','median','count','min','max'])
print(stats_rtt)
print(stats_ott_send)
print(stats_ott_recv)


              mean       var       std    median  count       min       max
Traffico                                                                   
0         0.025263  0.000026  0.005076  0.024877   3973  0.015088  0.079156
1         0.025283  0.000009  0.003010  0.025076   3972  0.015168  0.073851
2         0.025046  0.000005  0.002311  0.024981   3980  0.014761  0.077302
3         0.024747  0.000007  0.002621  0.024740   3988  0.014981  0.076113
              mean       var       std    median  count       min       max
Traffico                                                                   
0         0.010404  0.000006  0.002464  0.010213   3973  0.005192  0.060964
1         0.010672  0.000005  0.002316  0.010538   3972  0.005194  0.060283
2         0.010522  0.000002  0.001246  0.010514   3980  0.005904  0.053243
3         0.010314  0.000003  0.001609  0.010238   3988  0.005874  0.052584
              mean       var       std    median  count       min       max
Traffico    